In [1]:
from qh_circuit import *
from qh_gates import *
from tn_simu import TNArchitecture, TNSimulator, PEPSArchitecture
import qiskit
from qiskit import Aer

import matplotlib.pyplot as plt

In [2]:
def fidelity(sv1: np.ndarray, sv2: np.ndarray):
    sv1_normed = sv1 / np.linalg.norm(sv1) ** 2
    sv2_normed = sv2 / np.linalg.norm(sv2) ** 2
    return np.abs(np.inner(sv1_normed, sv2_normed.conj())) ** 2

In [3]:
supported_gates = list(gate_dict.keys())
print(f"Supported gates: {supported_gates}")

Supported gates: ['X', 'Y', 'Z', 'H', 'S', 'T', 'P', 'RX', 'RY', 'RZ', 'U', 'CX', 'CZ', 'RXX', 'RYY', 'RZZ']


In [4]:
def get_random_connected_qubit(arch: TNArchitecture, qubit):
    lst = np.array(arch.neighbours[qubit])
    qubit2 = int(np.random.choice(lst, 1))
    return qubit2

In [5]:
qc = QHCircuit(10)
for gate in supported_gates:
    try:
        dir(qc).index(gate.lower())
    except:
        print(f"Warning: Method {gate} is not defined in QHCircuit")

In [6]:
def check_one_qubit_gates(length=4, width=3, ngates=3, round=10):
    failed = []
    nqubits = length * width
    for _ in range(round):
        qhqc = QHCircuit(nqubits)
        qiskit_qc = qiskit.QuantumCircuit(nqubits)

        for _ in range(ngates):
            gate = np.random.randint(0, 10)
            qubit = np.random.randint(0, nqubits)
            if gate == 0: # x
                qhqc.x(qubit)
                qiskit_qc.x(qubit)
            elif gate == 1: # y
                qhqc.y(qubit)
                qiskit_qc.y(qubit)
            elif gate == 2: # z
                qhqc.z(qubit)
                qiskit_qc.z(qubit)
            elif gate == 3: # h
                qhqc.h(qubit)
                qiskit_qc.h(qubit)
            elif gate == 4: # s
                qhqc.s(qubit)
                qiskit_qc.s(qubit)
            elif gate == 5: # t
                qhqc.t(qubit)
                qiskit_qc.t(qubit)
            elif gate == 6: # p
                para = np.random.rand()
                qhqc.p(qubit, para)
                qiskit_qc.p(para, qubit)
            elif gate == 7: # rx
                para = np.random.rand()
                qhqc.rx(qubit, para)
                qiskit_qc.rx(para, qubit)            
            elif gate == 8: # ry
                para = np.random.rand()
                qhqc.ry(qubit, para)
                qiskit_qc.ry(para, qubit)            
            elif gate == 9: # rz
                para = np.random.rand()
                qhqc.rz(qubit, para)
                qiskit_qc.rz(para, qubit)

        peps_simulator = TNSimulator(PEPSArchitecture(length, width), qhqc, xi=1e-8, max_chi=2**ngates)
        qiskit_simulator = Aer.get_backend('statevector_simulator')

        peps_simulator.simulate()
        peps_sv = peps_simulator.get_all_amplitudes()
        qiskit_sv = qiskit_simulator.run(qiskit_qc).result().get_statevector().data

        fid = fidelity(peps_sv, qiskit_sv)
        if fid < 1 - 1e-3:
            failed.append(qhqc)
            print(f"Failed! Fidelity = {fid}")

    if len(failed) == 0:
        print(f"All successful")
    else:
        return failed 


In [7]:
failed = check_one_qubit_gates(length=4, width=2, ngates=100, round=10)

All successful


In [8]:
def check_cnot(length=4, width=3, ngates=3, round=10):
    nqubits = length * width
    failed = []
    arch = PEPSArchitecture(length, width)

    for _ in range(round):
        qhqc = QHCircuit(nqubits)
        qiskit_qc = qiskit.QuantumCircuit(nqubits)

        for _ in range(ngates):
            gate = np.random.randint(0, 6)
            qubit = np.random.randint(0, nqubits)
            if gate == 0: # h
                qhqc.h(qubit)
                qiskit_qc.h(qubit)
            elif gate == 1: # p
                para = np.random.rand()
                qhqc.p(qubit, para)
                qiskit_qc.p(para, qubit)
            elif gate == 2: # rx
                para = np.random.rand()
                qhqc.rx(qubit, para)
                qiskit_qc.rx(para, qubit)            
            elif gate == 3: # ry
                para = np.random.rand()
                qhqc.ry(qubit, para)
                qiskit_qc.ry(para, qubit)            
            elif gate == 4: # rz
                para = np.random.rand()
                qhqc.rz(qubit, para)
                qiskit_qc.rz(para, qubit)
            elif gate == 5: # cnot
                qubit2 = get_random_connected_qubit(arch, qubit)
                qhqc.cnot(qubit, qubit2)
                qiskit_qc.cnot(qubit, qubit2)

        peps_simulator = TNSimulator(arch, qhqc, xi=1e-8, max_chi=2**ngates)
        qiskit_simulator = Aer.get_backend('statevector_simulator')

        try:
            peps_simulator.simulate()
            peps_sv = peps_simulator.get_all_amplitudes()
            qiskit_sv = qiskit_simulator.run(qiskit_qc).result().get_statevector().data

            assert fidelity(peps_sv, qiskit_sv) > 1 - 1e-3
        except:
            print("Failed!")
            failed.append(qhqc)

    if len(failed) == 0:
        print(f"All successful")
    else:
        return failed 


In [9]:
failed = check_cnot(length=4, width=3, ngates=4, round=10)

All successful


In [10]:
# print(failed[0])

In [11]:
def check_two_qubit_gates(length=4, width=3, ngates=3, round=10):
    nqubits = length * width
    failed = []
    arch = PEPSArchitecture(length, width)
    for _ in range(round):
        qhqc = QHCircuit(nqubits)
        qiskit_qc = qiskit.QuantumCircuit(nqubits)

        for _ in range(ngates):
            gate = np.random.randint(0, 9)
            qubit = np.random.randint(0, nqubits)
            qubit2 = get_random_connected_qubit(arch, qubit)

            if gate == 0: # h
                qhqc.h(qubit)
                qiskit_qc.h(qubit)
            elif gate == 1: # p
                para = np.random.rand()
                qhqc.p(qubit, para)
                qiskit_qc.p(para, qubit)
            elif gate == 2: # rx
                para = np.random.rand()
                qhqc.rx(qubit, para)
                qiskit_qc.rx(para, qubit)            
            elif gate == 3: # ry
                para = np.random.rand()
                qhqc.ry(qubit, para)
                qiskit_qc.ry(para, qubit)            
            elif gate == 4: # rz
                para = np.random.rand()
                qhqc.rz(qubit, para)
                qiskit_qc.rz(para, qubit)
            elif gate == 5: # cnot
                qhqc.cnot(qubit, qubit2)
                qiskit_qc.cnot(qubit, qubit2)
            elif gate == 6: # rxx
                para = np.random.rand()
                qhqc.rxx(qubit, qubit2, para)
                qiskit_qc.rxx(para, qubit, qubit2)
            elif gate == 7: # ryy
                para = np.random.rand()
                qhqc.ryy(qubit, qubit2, para)
                qiskit_qc.ryy(para, qubit, qubit2)
            elif gate == 8: # rzz
                para = np.random.rand()
                qhqc.rzz(qubit, qubit2, para)
                qiskit_qc.rzz(para, qubit, qubit2)

        peps_simulator = TNSimulator(arch, qhqc, xi=1e-8, max_chi=2**ngates)
        qiskit_simulator = Aer.get_backend('statevector_simulator')

        peps_simulator.simulate()
        peps_sv = peps_simulator.get_all_amplitudes()
        qiskit_sv = qiskit_simulator.run(qiskit_qc).result().get_statevector().data

        fid = fidelity(peps_sv, qiskit_sv)
        if fid < 1 - 1e-3:
            failed.append(qhqc)
            print(f"Failed! Fidelity = {fid}")

    if len(failed) == 0:
        print(f"All successful")
    else:
        return failed 


In [12]:
failed = check_two_qubit_gates(length=4, width=3, ngates=4, round=20)

All successful
